In [1]:
#SparkContext.stop(sc)

In [2]:
import findspark
findspark.init()
import pyspark

In [3]:
import pandas as pd
import numpy as np
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
SparkContext.setSystemProperty('spark.executor.memory', '100g')
SparkContext.setSystemProperty('spark.driver.memory','300g')
sc = SparkContext('local',"recommendation")
sql_sc = SQLContext(sc)

In [ ]:
#import spark package
import pandas as pd
import numpy as np
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
conf = SparkConf().setAppName("App")
conf = (conf.setMaster('local[*]').set('spark.executor.memory', '140G').set('spark.driver.memory', '450G').set('spark.driver.maxResultSize', '140G'))
sc = SparkContext(conf=conf)
#SparkContext.setSystemProperty('spark.executor.memory', '10g')
#SparkContext.setSystemProperty('spark.driver.memory','30g')
#sc = SparkContext('local',appName="recommendation")
sql_sc = SQLContext(sc)

# Load and parse the data

In [4]:
#data=pd.read_csv(r"C:\Users\yvonn\OneDrive\Documents\WeChat Files\yifan729796133\Files\writeNewDataLog_2018-03-16\writeNewDataLog_2018-03-16.csv",sep='\t',encoding="utf-8")
#data1=pd.read_csv(r"C:\Users\yvonn\OneDrive\Documents\WeChat Files\yifan729796133\Files\writeNewDataLog20180315\writeNewDataLog20180315.csv",sep='\t',encoding="utf-8")
#data1=pd.read_csv(r"\data\data\writeNewDataLog_2018-03-19.csv",sep='\t',encoding="utf-8",usecols=['user_id_x','user_work_id','activity','score'])
#data1 = sql_sc.createDataFrame(data1)

data21=pd.read_csv("/data/data/writeNewDataLog_2018-03-27.csv",sep='\t',encoding="utf-8",usecols=['user_id_x','user_work_id','activity'])
data2 = sql_sc.createDataFrame(data21)
data31=pd.read_csv("/data/data/writeNewDataLog_2018-03-28.csv",sep='\t',encoding="utf-8",usecols=['user_id_x','user_work_id','activity','score'])
data3 = sql_sc.createDataFrame(data31)
data41=pd.read_csv("/data/data/writeNewDataLog_2018-03-29.csv",sep='\t',encoding="utf-8",usecols=['user_id_x','user_work_id','activity','score'])
data4 = sql_sc.createDataFrame(data41)
#data5=pd.read_csv("/data/data/writeNewDataLog_2018-03-30.csv",sep='\t',encoding="utf-8",usecols=['user_id_x','user_work_id','activity','score'])
#data5 = sql_sc.createDataFrame(data5)

In [5]:
from pyspark.sql.types import IntegerType
temp=data2.union(data3).union(data4)

In [6]:
#print out the columns
temp.printSchema()

root
 |-- user_id_x: long (nullable = true)
 |-- user_work_id: long (nullable = true)
 |-- activity: string (nullable = true)



In [7]:
#get distinct user_id and user_work_id
temp.select('user_id_x').distinct().count()

42635

In [8]:
#temp.select('user_work_id').distinct().count()

In [9]:
#change dataframe to a table
temp.registerTempTable('df')

In [10]:
#use sql command to deal with table defined
sqlContext = SQLContext(sc)
temp=sqlContext.sql("select *,(case when activity = '送礼物' then 12 when activity='评论' then 10 when activity='播放' then 2 when activity='下载' then 6 end) as score from df")

In [11]:
sqlContext.dropTempTable("df")

In [12]:
data=temp.groupby(['user_id_x','user_work_id']).agg({'score': 'sum'})
data.show(5)

+---------+------------+----------+
|user_id_x|user_work_id|sum(score)|
+---------+------------+----------+
|295126169|   319859067|       638|
|292972921|   322883555|       314|
|288151290|   322063288|       502|
| 81112189|   317886919|       102|
|297355795|   322883561|       306|
+---------+------------+----------+
only showing top 5 rows



In [13]:
data = data.withColumnRenamed("sum(score)", "score")

In [14]:
data.registerTempTable('data')
sqlContext.sql("select user_id_x,user_work_id,score from data").show(5)

+---------+------------+-----+
|user_id_x|user_work_id|score|
+---------+------------+-----+
|295126169|   319859067|  638|
|292972921|   322883555|  314|
|288151290|   322063288|  502|
| 81112189|   317886919|  102|
|297355795|   322883561|  306|
+---------+------------+-----+
only showing top 5 rows



In [28]:
from pyspark.sql.functions import col
user=data.groupby('user_id_x').count().withColumnRenamed("count", "n").filter("n >= 10").select('user_id_x')
data1=data.join(user,['user_id_x'])
work=data1.groupby('user_work_id').count().withColumnRenamed("count", "n").filter("n >= 10").select('user_work_id')
data2=data1.join(work,['user_work_id'])


In [29]:
data2.show(5)

+------------+---------+-----+
|user_work_id|user_id_x|score|
+------------+---------+-----+
|   115833693| 83279549|  104|
|   115833693|287081204|  130|
|   115833693|296585419|  102|
|   115833693| 52682528|  102|
|   115833693| 81894225|  102|
+------------+---------+-----+
only showing top 5 rows



In [45]:
#data.columns=['user_id', 'item_id', 'rating']
df=data2
from pyspark.sql.functions import *
from pyspark.sql.window import Window

ranked =  df.withColumn("rank", dense_rank().over(Window.orderBy("user_id_x")))
ranked1=ranked.withColumn("rank1",dense_rank().over(Window.orderBy("user_work_id")))

In [46]:
df=ranked1.select('rank','rank1','score').withColumnRenamed("rank", "user_id_x").withColumnRenamed("rank1","user_work_id",)

In [47]:
ranked.select('rank').rdd.max()[0]

7111

In [48]:
df.show(5)

+---------+------------+-----+
|user_id_x|user_work_id|score|
+---------+------------+-----+
|      156|           1|    2|
|      978|           1|  202|
|     1276|           1|  202|
|     1288|           1|  202|
|     2583|           1|    4|
+---------+------------+-----+
only showing top 5 rows



In [60]:
training_RDD, test_RDD = df.randomSplit([0.8, 0.2])
test_for_predict_RDD = test_RDD.rdd.map(lambda x: (x[0], x[1]))

In [59]:
from pyspark.mllib.recommendation import Rating
from  pyspark.mllib.recommendation import ALS
from pyspark.mllib.recommendation import MatrixFactorizationModel
from sklearn.metrics import mean_squared_error
import itertools,math
test_for_predict_RDD = df.rdd.map(lambda x: (x[0], x[1]))
sc.setCheckpointDir('checkpoint/')
#rdd1 = sc.parallelize(ratings)
#rdd = sql_sc.createDataFrame(ratings)
#testrdd = sc.parallelize(test)
#rdd2 = rdd1.map(lambda x: [int(i) for i in x])
#df = rdd2.toDF()
ranks=[20,60,100]
lambdas=[0.2,0.02,0.002]
iterations=[5,8,10]
for rank, lmbda, numIter in itertools.product(ranks,
                                              lambdas,
                                              iterations):
    model = ALS.train(df, rank, numIter, lmbda)
    predictions=model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = df.rdd.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    print(error)

8.312046139671208
5.586589320360797
4.716263686889607
9.090257774314388
5.686783011160867
4.680984604051102
9.804655026239024
6.814099204499685
5.655776931261197
1.3957310421961147
0.8669901762587191
0.7021991828106324
1.162971409368757
0.61751725493901
0.5099146979688567
0.6810634872258862
0.42137206209163103
0.35511065921144364
0.7072047622923782
0.562969783009437
0.49071474293718154
0.503201449540762
0.23130795758928244
0.2189607459858207
0.21012398673822666
0.1532530377693966
0.14479691377132708


In [64]:
from pyspark.mllib.recommendation import Rating
from  pyspark.mllib.recommendation import ALS
from pyspark.mllib.recommendation import MatrixFactorizationModel
from sklearn.metrics import mean_squared_error
import itertools,math
test_for_predict_RDD = test_RDD.rdd.map(lambda x: (x[0], x[1]))
sc.setCheckpointDir('checkpoint/')
#rdd1 = sc.parallelize(ratings)
#rdd = sql_sc.createDataFrame(ratings)
#testrdd = sc.parallelize(test)
#rdd2 = rdd1.map(lambda x: [int(i) for i in x])
#df = rdd2.toDF()
ranks=[100,200]
lambdas=[0.2,0.02,0.002]
iterations=[5,8,10]
for rank, lmbda, numIter in itertools.product(ranks,
                                              lambdas,
                                              iterations):
    model = ALS.train(training_RDD, rank, numIter, lmbda)
    predictions=model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = test_RDD.rdd.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
    print(error)

473.9801713954781
473.27342459816373
469.3410918172375
482.2407211085714
475.66974162095113
465.68261119342935
490.5522598887366
473.65153493334833
464.56627664362355
472.73578003936194
471.7094698739519
465.77249160787693
474.9631887239488
463.5805762346791
467.16654834640286
455.6525955673623
473.57775829891835
469.13419642400066
